# Практическое занятие №2

## Фильтрация изображений

Подключаем необходимые либы

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

Прежде, чем фильтровать изображение посмотрим еще один тип шума: высокочастотный

In [ ]:
fileName = 'Mandrill.png'
img = cv2.imread(fileName)
if len(img.shape) > 2:
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

plt.imshow(img, cmap='gray')

Генерируем шум

In [ ]:
M, N = img.shape

In [ ]:
# Генерируем случайную частоту
f = np.random.randint(1, 5000)
# Выбираем случайную частоту дискретизации (> 2*f)
fs = np.random.randint(2, 10) * f
# Считаем период дискретизации
ts = 1/fs
t = np.arange(0, M*N, 1)
# Дискретизируем сигнал
T = np.array(t) * ts

In [ ]:
# Получаем зашумленный сигнал
dirty_cos = np.cos(2 * np.pi * f * T) * 255
# Перегоняем его в размер изображения
dirty_cos = np.reshape(dirty_cos, (M, N)).astype('uint8')

plt.imshow(dirty_cos, cmap='gray')

Добавим полученный шум к изображению

In [ ]:
plt.hist(dirty_cos.ravel(), bins=256)
plt.show()

In [ ]:
hf_img = cv2.add(img, dirty_cos)
plt.imshow(hf_img, cmap='gray')

Теперь переходим к фильтрации изображений

Сначала попробуем отфильтровать импульсный шум

Для этого будем использовать скользящую медиану

Попробуем отфильтровать наш ВЧ-шум при помощи скользящего среднего

Поймем, что выходит не очень. И будем использовать другой способ

Для начала посмотрим двух-мерный Фурье-спектр

In [ ]:
M, N = hf_img.shape

fft2 = np.fft.fft2(hf_img)
ft = np.fft.fftshift(fft2)

plt.imshow(np.log(np.abs(ft)), cmap='gray')

Видим те части, которые нам нужно отфильтровать

In [ ]:
N2 = int(N/2)
M2 = int(M/2)

ft[:M2 - 25, N2 - 2: N2 + 2] = 1
ft[M2 + 25:, N2 - 2: N2 + 2] = 1

plt.imshow(np.log(np.abs(ft)), cmap='gray')

После того, как мы отфильтровали нужные области, возвращаем изображению нормальный вид

In [ ]:
restored_img = abs(np.fft.ifft2(ft))

plt.imshow(restored_img, cmap='gray')

## Другие операции с изображениями

Необходима библиотека scikit-image

In [ ]:
from skimage.exposure import rescale_intensity

In [ ]:
def convolve(image, kernel):
	# считываем размеры изображения и ядра свертки
	(iH, iW) = image.shape[:2]
	(kH, kW) = kernel.shape[:2]
	# делаем паддинг для изображения
	pad = (kW - 1) // 2
	image = cv2.copyMakeBorder(image, pad, pad, pad, pad,
		cv2.BORDER_REPLICATE)
    # создаем пустое изображение, в которое будем записывать результат
	output = np.zeros((iH, iW), dtype="float32")
    # итерируемся по x и y
	for y in np.arange(pad, iH + pad):
		for x in np.arange(pad, iW + pad):
			# выхватываем окно
			roi = image[y - pad:y + pad + 1, x - pad:x + pad + 1]
			# и сворачиваем его с ядром
			k = (roi * kernel).sum()
			output[y - pad, x - pad] = k
    # rescale the output image to be in the range [0, 255]
	output = rescale_intensity(output, in_range=(0, 255))
	output = (output * 255).astype("uint8")
	# return the output image
	return output

In [ ]:
# шарпенинг (повышение резкости)
sharpen = np.array((
	[0, -1, 0],
	[-1, 5, -1],
	[0, -1, 0]), dtype="int")

# выделение границ
laplacian = np.array((
	[0, 1, 0],
	[1, -4, 1],
	[0, 1, 0]), dtype="int")

sobelX = np.array((
	[-1, 0, 1],
	[-2, 0, 2],
	[-1, 0, 1]), dtype="int")

sobelY = np.array((
	[-1, -2, -1],
	[0, 0, 0],
	[1, 2, 1]), dtype="int")

prewittX = np.array((
	[-1, 0, 1],
	[-1, 0, 1],
	[-1, 0, 1]), dtype="int")

prewittY = np.array((
	[-1, -1, -1],
	[0, 0, 0],
	[1, 1, 1]), dtype="int")

In [ ]:
result = convolve(img, laplacian)

plt.imshow(result, cmap='gray')

Эквализация гистограммы

$ 255 * \left( \frac{I - I_{min}}{I_{max}-I_{min}} \right) $

In [ ]:
plt.hist(img.ravel(), 256, [0, 255])
plt.show()

Imin = 10
Imax = 250

img_eq = 255 * ((img - Imin)/(Imax - Imin))

plt.hist(img_eq.ravel(), 256, [0, 255])
plt.show()

plt.imshow(img_eq, cmap='gray')

Гамма-коррекция

$ \left( \Big(\frac{I}{255}\Big)^{\frac{1}{\gamma}} \right)*255 $

In [ ]:
gamma = 0.7
invGamma = 1 / gamma

table = [((i / 255) ** invGamma) * 255 for i in range(256)]
table = np.array(table, np.uint8)

img_c = cv2.LUT(img, table)

plt.imshow(img_c, cmap='gray')

Бинаризация

In [ ]:
thr = 10
img_bin = (img > thr) * img

plt.imshow(img_bin, cmap='gray')